In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import db_utils, sqlalchemy
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import seaborn as sns
from scipy.stats import fisher_exact
from scipy import stats
import math
import os
import gzip
import re
import glob
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import math
import requests
import time
connect = db_utils.get_connection()

In [3]:
# Define a function that fetches the AA sequence of a protein based on its UniProt ID using the UniProt API
def fetch_protein_sequence(uniprot_accession):
    base_url = "https://www.uniprot.org/uniprot/"
    response_format = ".fasta"

    # Construct the URL to fetch the sequence
    url = f"{base_url}{uniprot_accession}{response_format}"

    try:
        # Send a GET request to UniProt to fetch the protein sequence
        response = requests.get(url)

        # Check if the request was successful (HTTP status code 200)
        if response.status_code == 200:
            # The sequence is present in the response text
            if response.text == '':
                print(f"Error: Unable to fetch sequence. Possibly obsolete UniProt ID!")
                return None
            else:
                sequence = response.text.split('\n', 1)[1].replace('\n', '')
                return sequence
        else:
            print(f"Error: Unable to fetch sequence. HTTP status code {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print("Error: Request failed.")
        print(str(e))
        return None

In [4]:
# Load the manual curation dataset as a DataFrame
df_mc = pd.read_csv('/Users/johgeist/Downloads/DDI_curation table_wihtout_homo_protein_structures_4.csv')

In [5]:
df_mc = df_mc[~df_mc['DDI_type'].str.contains('curation')]

In [6]:
df_mc['PDB_ID'] = df_mc['PDB_ID'].str.upper()

In [17]:
# Connect to MySQL and load in the preprocessed 3did database as a DataFrame
eng = sqlalchemy.create_engine('mysql://', creator= db_utils.get_connection)
query= pd.read_sql_query('''select * from chopyan_db.3did_agg_DDI_PDB''', con= eng)
DDI_CS = pd.DataFrame(query)

/var/folders/gv/nb7y89n11zb14h3c0y1b594w0000gn/T/ipykernel_58205/1863732494.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  DDI_CS= pd.read_sql_query('''select * from chopyan_db.3did_agg_DDI_PDB''', con= connect)


In [18]:
DDI_CS

,DDI_type,PDB_ID,Chain_ID1,DomainID1,DomainName1,DomainStart1,DomainEnd1,Chain_ID2,DomainID2,DomainName2,DomainStart2,DomainEnd2,Score,Zscore,Chain1Iface,Chain2Iface,isInterchain
0,PF00001_PF00001,1gzm,A,PF00001,7tm_1,54,306,B,PF00001,7tm_1,54,306,26.18,7.82923,"E:196,E:196,E:196,E:196,E:197,N:199,E:201,E:20...","F:228,K:231,E:232,A:233,E:232,F:228,V:227,F:22...",1
1,PF00001_PF00001,1hzx,B,PF00001,7tm_1,54,246,B,PF00001,7tm_1,224,306,42.49,7.03413,"I:54,I:54,I:54,N:55,N:55,N:55,T:58,L:72,L:72,L...","V:300,P:303,V:304,A:299,V:300,P:303,Y:306,V:25...",0
2,PF00001_PF00001,1l9h,B,PF00001,7tm_1,54,246,B,PF00001,7tm_1,224,306,40.17,9.39410,"I:54,I:54,N:55,N:55,N:55,T:58,L:72,L:72,L:72,L...","V:300,P:303,A:299,V:300,P:303,Y:306,V:250,M:25...",0
3,PF00001_PF00001,2j4y,A,PF00001,7tm_1,54,306,B,PF00001,7tm_1,54,306,26.85,6.86878,"N:199,E:201,E:201,E:201,S:202,S:202,I:205,V:20...","Q:236,F:228,K:231,E:232,F:228,E:232,F:228,F:22...",1
4,PF00001_PF00001,2vt4,C,PF00001,7tm_1,58,343,D,PF00001,7tm_1,58,343,20.02,6.36751,"L:152,V:160,T:164,T:164,A:167,I:168,I:168,A:17...","F:315,A:206,A:206,A:210,W:181,A:210,I:214,W:18...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235443,PF18841_PF17433,1wmr,A,PF18841,B_solenoid_dext,321,354,A,PF17433,Glyco_hydro_49N,19,184,9.52,4.52235,"D:341,D:341,D:341,K:343,K:343,V:345,V:345,V:34...","R:153,S:182,P:183,S:101,S:182,Q:97,W:99,L:178,...",0
235444,PF18841_PF17433,1x0c,B,PF18841,B_solenoid_dext,321,354,B,PF17433,Glyco_hydro_49N,19,184,9.52,4.45403,"D:341,D:341,D:341,K:343,K:343,V:345,V:345,V:34...","R:153,S:182,P:183,S:101,S:182,Q:97,W:99,L:178,...",0
235445,PF18841_PF17433,2z8g,A,PF18841,B_solenoid_dext,321,354,A,PF17433,Glyco_hydro_49N,19,184,9.24,4.49346,"D:341,D:341,D:341,K:343,K:343,V:345,V:345,V:34...","R:153,S:182,P:183,S:101,S:182,Q:97,W:99,L:178,...",0
235446,PF18841_PF17433,3wwg,C,PF18841,B_solenoid_dext,321,354,C,PF17433,Glyco_hydro_49N,19,184,9.24,4.61412,"D:341,D:341,D:341,K:343,K:343,V:345,V:345,V:34...","R:153,S:182,P:183,S:101,S:182,Q:97,W:99,L:178,...",0


In [20]:
DDI_CS.rename({'Chain1Sequence': 'Chain1Sequence_3did', 'Chain2Sequence': 'Chain2Sequence_3did'}, inplace=True, axis=1)

In [21]:
DDI_CS['PDB_ID']= DDI_CS['PDB_ID'].str.upper()

In [28]:
DDI_CS

,DDI_type,PDB_ID,Chain_ID1,DomainID1,DomainName1,DomainStart1,DomainEnd1,Chain_ID2,DomainID2,DomainName2,DomainStart2,DomainEnd2,Score,Zscore,Chain1Iface,Chain2Iface,isInterchain
0,PF00001_PF00001,1GZM,A,PF00001,7tm_1,54,306,B,PF00001,7tm_1,54,306,26.18,7.82923,"E:196,E:196,E:196,E:196,E:197,N:199,E:201,E:20...","F:228,K:231,E:232,A:233,E:232,F:228,V:227,F:22...",1
1,PF00001_PF00001,1HZX,B,PF00001,7tm_1,54,246,B,PF00001,7tm_1,224,306,42.49,7.03413,"I:54,I:54,I:54,N:55,N:55,N:55,T:58,L:72,L:72,L...","V:300,P:303,V:304,A:299,V:300,P:303,Y:306,V:25...",0
2,PF00001_PF00001,1L9H,B,PF00001,7tm_1,54,246,B,PF00001,7tm_1,224,306,40.17,9.39410,"I:54,I:54,N:55,N:55,N:55,T:58,L:72,L:72,L:72,L...","V:300,P:303,A:299,V:300,P:303,Y:306,V:250,M:25...",0
3,PF00001_PF00001,2J4Y,A,PF00001,7tm_1,54,306,B,PF00001,7tm_1,54,306,26.85,6.86878,"N:199,E:201,E:201,E:201,S:202,S:202,I:205,V:20...","Q:236,F:228,K:231,E:232,F:228,E:232,F:228,F:22...",1
4,PF00001_PF00001,2VT4,C,PF00001,7tm_1,58,343,D,PF00001,7tm_1,58,343,20.02,6.36751,"L:152,V:160,T:164,T:164,A:167,I:168,I:168,A:17...","F:315,A:206,A:206,A:210,W:181,A:210,I:214,W:18...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235443,PF18841_PF17433,1WMR,A,PF18841,B_solenoid_dext,321,354,A,PF17433,Glyco_hydro_49N,19,184,9.52,4.52235,"D:341,D:341,D:341,K:343,K:343,V:345,V:345,V:34...","R:153,S:182,P:183,S:101,S:182,Q:97,W:99,L:178,...",0
235444,PF18841_PF17433,1X0C,B,PF18841,B_solenoid_dext,321,354,B,PF17433,Glyco_hydro_49N,19,184,9.52,4.45403,"D:341,D:341,D:341,K:343,K:343,V:345,V:345,V:34...","R:153,S:182,P:183,S:101,S:182,Q:97,W:99,L:178,...",0
235445,PF18841_PF17433,2Z8G,A,PF18841,B_solenoid_dext,321,354,A,PF17433,Glyco_hydro_49N,19,184,9.24,4.49346,"D:341,D:341,D:341,K:343,K:343,V:345,V:345,V:34...","R:153,S:182,P:183,S:101,S:182,Q:97,W:99,L:178,...",0
235446,PF18841_PF17433,3WWG,C,PF18841,B_solenoid_dext,321,354,C,PF17433,Glyco_hydro_49N,19,184,9.24,4.61412,"D:341,D:341,D:341,K:343,K:343,V:345,V:345,V:34...","R:153,S:182,P:183,S:101,S:182,Q:97,W:99,L:178,...",0


In [26]:
# Load in addtional protein information (generated before) and merge with DDI_CS dataframe
df_homo_protein = pd.read_csv('/Users/johgeist/Downloads/DDI_type_structure_level_interchain_homo_protein_whole_3did.csv', index_col=0)
df_homo_protein = df_homo_protein.drop(columns=['IsInterchain'])

In [32]:
DDI_CS = pd.merge(DDI_CS, df_homo_protein, on= ['DDI_type', 'PDB_ID'])

In [34]:
# Select one structure and interface per DDI type by selecting the one with the highest score in 3did
df_Score = pd.merge(DDI_CS, df_mc, on= ['DDI_type', 'PDB_ID'], how = 'inner')
df_Score_index = df_Score.groupby('DDI_type').agg({'Score': 'idxmax'})
df_Score_max = df_Score[df_Score.index.isin(df_Score_index.Score.tolist())]

In [38]:
# Using the UniProtID the interacting proteins carrying a DDI in the structure with the highest 3did score, fetch
# fetch the respective sequence using the UniProt API
seq_1 = []
seq_2 = []
for UniProt1, UniProt2 in zip(df_Score_max['UniProt1'].tolist(), df_Score_max['UniProt2'].tolist()):
    print(UniProt1)
    protein_sequence1 = fetch_protein_sequence(UniProt1)
    print(UniProt2)
    protein_sequence2 = fetch_protein_sequence(UniProt2)
    seq_1.append(protein_sequence1)
    seq_2.append(protein_sequence2)
#     print('Sleeping 1 second to not flood the server')
#     time.sleep(1)

P62193
P21670
Q8U082
Q8U3I5
P61244
P01106
P12643
Q92859
O75832
No UniProt
Error: Unable to fetch sequence. HTTP status code 400
No UniProt
Error: Unable to fetch sequence. HTTP status code 400
P19235
P00791
P19400
P07897
Q05546
P00772
P19957
P63081
Q6AXS4
P62136
O35274
A8IKR9
A8I2U9
Q6ZKC0
Q93WI9
No UniProt
Error: Unable to fetch sequence. HTTP status code 400
No UniProt
Error: Unable to fetch sequence. HTTP status code 400
P25791
P01764
Q8BTI9
O08908
P35222
O00482
O29915
No UniProt
Error: Unable to fetch sequence. HTTP status code 400
Q9JK83
P60953
P23004
P00129
Q53H12
Q9BSF4
O60493
O75436
P78348
P0DKR6
P00363
P0AC47
Q09057
P02787
P19919
P19921
Q5SIT6
P74901
O15226
O43143
P38238
Q08924
P56159
P07949
P40933
P14784
P12004
P39748
Q15796
O95405
Q1HPP4
H9JHM9
E2RTU1
V6TR15
P53290
Q02205
W5Q5U7
No UniProt
Error: Unable to fetch sequence. HTTP status code 400
Q9Y294
P62805
O43741
P54645
Q12158
P32908
Q91349
K9ZTJ6
P32910
P36121
Q9CR61
Q9D6J5
P35178
P10962
P30533
Q07954
Q4K4F7
Q9I6T0
P55081
Q

In [39]:
# add the sequences of each protein carrying a domain as addtional information to the grouped 3did DataFrame
# df_Score_max
df_Score_max = df_Score_max.copy()
df_Score_max['Uniprot_seq1']= seq_1
df_Score_max['Uniprot_seq2']= seq_2

/var/folders/gv/nb7y89n11zb14h3c0y1b594w0000gn/T/ipykernel_58205/1804662463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Score_max['Uniprot_seq1']= seq_1
/var/folders/gv/nb7y89n11zb14h3c0y1b594w0000gn/T/ipykernel_58205/1804662463.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Score_max['Uniprot_seq2']= seq_2


In [40]:
df_Score_max

,DDI_type,PDB_ID,Chain_ID1,DomainID1,DomainName1,DomainStart1,DomainEnd1,Chain_ID2,DomainID2,DomainName2,...,Motif_binding_domain,DMI_overlap_checked,DMI_overlap_identified,DDI_approved,Curator,Comments,non-approved category,Unnamed: 19,Uniprot_seq1,Uniprot_seq2
0,PF00004_PF10584,6EPF,I,PF00004,AAA,222,355,C,PF10584,Proteasome_A_N,...,NaN,NaN,NaN,0.0,"JG, KL",1 out of 1 supporting CS showed an interchain ...,HMM (at least one Pfam has unfolded structure),linear stretch within domain,MGQSQSGGHGPGGGKKDDKDKKKKYEPPVPTRVGKKKKKTKGPDAA...,MSRRYDSRTTIFSPEGRLYQVEYAMEAIGHAGTCLGILANDGVLLA...
1,PF00009_PF01873,2D74,A,PF00009,GTP_EFTU,9,201,B,PF01873,eIF-5_eIF-2B,...,NaN,NaN,NaN,1.0,JG,11 out of 11 supporting CS showed interchain i...,NaN,NaN,MGEKRKTRQAEVNIGMVGHVDHGKTTLTKALTGVWTDTHSEELRRG...,MEIDYYDYEKLLEKAYQELPENVKHHKSRFEVPGALVTIEGNKTII...
3,PF00010_PF02344,6G6L,B,PF00010,HLH,214,254,C,PF02344,Myc-LZ,...,NaN,NaN,NaN,0.0,"JG, KL",2 out of 6 supporting structures in 3did showe...,crystal contact,NaN,MSDNDDIEVESDEEQPRFQSAADKRAHHNALERKRRDHIKDSFHSL...,MDFFRVVENQQPPATMPLNVSFTNRNYDLDYDSVQPYFYCDEEENF...
4,PF00019_PF00041,4UI2,B,PF00019,TGF_beta,294,395,A,PF00041,fn3,...,NaN,NaN,NaN,0.0,JG,Only 1 supporting CS was present for the DDI t...,crystal contact,study describes crystal contact interface,MVAGTRCLLALLLPQVLLGGAAGLVPELGRRKFAAASSGRPSSQPS...,MAAERGARRLLSTPSFWLYCLLLLGRRAPGAAAARSGSAPQSPGAS...
5,PF00023_PF07686,4NIK,A,PF00023,Ank,105,137,B,PF07686,V-set,...,PF00023,1.0,0.0,0.0,JL,Only one supporting structure was present in 3...,synthetic construct,NaN,MEGCVSNLMVCNLAYSGKLEELKESILADKSLATRTDQDSRTALHW...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,PF16004_PF00400,6ID0,C,PF16004,EFTUD2,56,110,T,PF00400,WD40,...,PF16004,1.0,0.0,0.0,"JG, KL",10 out of 10 supporting CS displayed interchai...,HMM (at least one Pfam has unfolded structure),linear stretch within the domain (whole domain...,MDTDLYDEFGNYIGPELDSDEDDDELGRETKDLDEMDDDDDDDDVG...,MVEEVQKHSVHTLVFRSLKRTHDMFVADNGKPVPLDEESHKRKMAI...
90,PF17292_PF08644,4KHB,H,PF17292,POB3_N,5,97,G,PF08644,SPT16,...,NaN,NaN,NaN,0.0,"JG, KL",3 out of 3 supporting CS displayed interchain ...,contact is minimal,NaN,MAAIESFDHIYLDLSKEPGKCRFAENGLGWKPVGGGETFTLDVSNI...,MADIKIDSKLFQERISHFISAWKADKRSGDALFGGVSSIVILMGKV...
92,PF17820_PF16523,3L4F,D,PF17820,PDZ_6,701,755,A,PF16523,betaPIX_CC,...,NaN,NaN,NaN,0.0,"JG, JL",Only one structure supporting the DDI type (Sc...,HMM (at least one Pfam has unfolded structure),motif is C-terminally included in the Pfam match,MTHSPATSEDEERHSASECPEGGSESDSSPDGPGRGPQGTRGRGSG...,MTDNANSQLVVRAKFNFQQTNEDELSFSKGDVIHVTRVEEGGWWEG...
93,PF17838_PF00071,3KZ1,B,PF17838,PH_16,950,1079,E,PF00071,Ras,...,NaN,NaN,NaN,1.0,JG,9 out of 9 supporting CS displayed interchain ...,NaN,NaN,MSVRLPQSIDRLSSLSSLGDSAPERKSPSHHRQPSDASETTGLVQR...,MAAIRKKLVIVGDGACGKTCLLIVFSKDQFPEVYVPTVFENYVADI...


In [1]:
# Check how many cases there are where no sequnece could be fetched
df_Score_max[(df_Score_max['Uniprot_seq1'].isna())| (df_Score_max['Uniprot_seq2'].isna())]

NameError: name 'df_Score_max' is not defined

In [43]:
df_Score_max.to_csv('/Users/johgeist/Downloads/new_protein_sequences_for_IUPred_for_John_new.csv')